<a href="https://colab.research.google.com/github/daystram/ml-playground/blob/master/04_gym_pendulum/04a_gym_pendulum_dnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 04a Gym Pendulum using DNN


In [0]:
!rm -r ml-playground > /dev/null 2>&1
!git clone https://github.com/daystram/ml-playground.git
!sh ml-playground/helper/setup.sh > /dev/null 2>&1

Cloning into 'ml-playground'...
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 128 (delta 69), reused 64 (delta 27), pack-reused 0
Receiving objects: 100% (128/128), 283.80 KiB | 9.15 MiB/s, done.
Resolving deltas: 100% (69/69), done.


In [0]:
import sys
sys.path.append('/content/ml-playground')

In [0]:
import gym

env = gym.make("Pendulum-v0")
print(env.observation_space)
print(env.action_space)

Box(3,)
Box(1,)


In [0]:
import numpy as np
from tqdm import tqdm_notebook

acc_fitness = []
training_data = []

sample_count = 20000
steps = 20000
goal_tres = -800

for i in tqdm_notebook(range(sample_count), desc="Generating"):
  fitness = 0
  action_seq = []
  prev_observation = env.reset()
  
  for _ in range(steps):
    action = np.random.uniform(-2, 2)
    action_seq.append([prev_observation, action])
    prev_observation, reward, done, _ = env.step([action])
    fitness += reward
    if done: break
  if fitness > goal_tres:
    acc_fitness.append(fitness)
    for seq in action_seq:
      training_data.append([seq[0], seq[1]/2])
env.close()
print("\n{} samples".format(len(training_data)))
print(np.array(acc_fitness))



148200 samples
[-757.38346866 -738.27686451 -778.79136609 -758.73300144 -771.32867102
 -744.32825291 -769.46263192 -759.42893151 -792.16256293 -767.54619381
 -743.6745997  -748.31451182 -782.17159041 -744.29948237 -757.3866011
 -775.24162285 -637.11680066 -709.02265909 -751.8682391  -793.64611667
 -794.17320879 -750.10387209 -767.46078848 -720.06401358 -637.91558206
 -763.81412036 -750.56140906 -753.57813411 -770.25408495 -644.16449999
 -790.91114206 -755.11082601 -773.03024661 -755.86395571 -766.38302195
 -727.92575421 -768.34164233 -752.37860152 -754.41681263 -766.73591013
 -784.93900528 -748.77375964 -758.86435199 -787.95976153 -767.42390056
 -765.20003331 -757.86772437 -770.04295612 -791.66299184 -794.385018
 -779.08492421 -783.79122804 -760.88219533 -791.05014637 -701.63386286
 -774.63350183 -772.64449212 -776.14179467 -641.30768578 -761.98771422
 -776.30323865 -756.43064952 -759.34530533 -761.37448877 -781.62025199
 -642.90188712 -768.71410793 -765.98853971 -780.76802487 -765.7

In [0]:
import gym

env = gym.make("Pendulum-v0")
print(env.observation_space)
print(env.observation_space.high)
print(env.observation_space.low)

print(env.action_space)
print(env.action_space.high)
print(env.action_space.low)

Box(3,)
[1. 1. 8.]
[-1. -1. -8.]
Box(1,)
[2.]
[-2.]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

drop_rate = 0.2
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(3,)))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(512, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(256, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(128, activation='relu'))
model.add(Dropout(drop_rate))
model.add(Dense(1, activation='tanh'))

model.compile(loss='mse',
              optimizer='sgd',
              metrics=['accuracy'])

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
X = np.array([i[0] for i in training_data])
y = np.array([i[1] for i in training_data])

model.fit(X, y, epochs=5, batch_size=32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/5
148200/148200 [==============================] - 31s 206us/step - loss: 0.3349 - acc: 0.0000e+00
Epoch 2/5
148200/148200 [==============================] - 31s 209us/step - loss: 0.3324 - acc: 0.0000e+00
Epoch 3/5
148200/148200 [==============================] - 32s 214us/step - loss: 0.3321 - acc: 0.0000e+00
Epoch 4/5
148200/148200 [==============================] - 31s 209us/step - loss: 0.3320 - acc: 0.0000e+00
Epoch 5/5
148200/148200 [==============================] - 31s 208us/step - loss: 0.3319 - acc: 0.0000e+00


In [0]:
from helper import nbdisplay as disp

!rm -r video
env = disp.wrap_env(gym.make("Pendulum-v0"))
observation = env.reset()
fitness = 0
  
for _ in range(steps):
  action = model.predict(np.array([observation]))
  print(action)
  print(observation)
  observation, reward, done, _ = env.step(list(action[0] * 2))
  fitness += reward
  if done: break

env.close()
disp.show_video()
print("Video can be viewed in Google Colab, use the link on top of the page to open")
print("------ Agent: Reward {}".format(fitness))  

xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


[[-0.00133101]]
[-0.80501994 -0.59324775 -0.66792347]
[[-0.00318719]]
[-0.836778   -0.54754231 -1.11325858]
[[-0.00393346]]
[-0.8760532  -0.48221447 -1.52487147]
[[-0.00454042]]
[-0.91760043 -0.39750402 -1.88771236]
[[-0.00461915]]
[-0.95550331 -0.29498039 -2.18720251]
[[-0.00447153]]
[-0.98403223 -0.17799036 -2.40982355]
[[-0.00467255]]
[-0.99866329 -0.05168789 -2.54465777]
[[-0.00474038]]
[-0.99699603  0.07745269 -2.58482546]
[[-0.0046616]]
[-0.97927657  0.20252752 -2.52815805]
[[-0.00436296]]
[-0.94834416  0.31724337 -2.37766089]
[[-0.00375202]]
[-0.90901862  0.4167555  -2.14103725]
[[-0.00282985]]
[-0.86714613  0.49805381 -1.82959623]
[[-0.00199693]]
[-0.82859765  0.55984457 -1.45690483]
[[-0.00131512]]
[-0.79845047  0.6020605  -1.03762049]
[[-0.00051058]]
[-0.78045524  0.62521166 -0.58646965]
[[0.00135904]]
[-0.77676193  0.62979434 -0.11771407]
[[0.00266191]]
[-0.78781904  0.61590678  0.35503939]
[[0.00283148]]
[-0.812337    0.5831883   0.81776805]
[[0.00323536]]
[-0.84733605  0.5

Video can be viewed in Google Colab, use the link on top of the page to open
------ Agent: Reward -1532.410013631935
